### Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import json
import random

#%matplotlib notebook
%load_ext autoreload
%autoreload 2

### Load Data

In [2]:
with open('sources-named-0-dataset.json', 'r') as dsFile:
    dataset = json.loads(dsFile.read())
    
random.shuffle(dataset)

In [3]:
X = np.array([np.array(x[0]) for x in dataset])
y = np.array([np.array(x[1]) for x in dataset])

print(X.shape)
print(y.shape)

(218, 128, 96)
(218,)


In [4]:
n_test = 50
split_pos = len(X) - n_test

train_X = X[:split_pos]
train_y = y[:split_pos]
test_X = X[split_pos:]
test_y = y[split_pos:]

print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(168, 128, 96)
(50, 128, 96)
(168,)
(50,)


In [5]:
from tensorflow.keras.utils import to_categorical
num_classes = 11
train_y = to_categorical(train_y, num_classes)
test_y = to_categorical(test_y, num_classes)

Distribution of Target

In [ ]:
plt.hist(train_y)

In [ ]:
# ensure the X and y line up
n = 30
plt.imshow(train_X[n], cmap='gray')
print(train_y[n])

### Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=15, 
                             horizontal_flip=True,
                             brightness_range=(0.7, 1.3),
                             fill_mode = 'mirror',
                             rescale=1.0/256.0)

test_datagen = ImageDataGenerator(rescale=1.0/256.0)

train_generator = datagen.flow(np.expand_dims(train_X, 3), train_y, batch_size=32)
test_generator = datagen.flow(np.expand_dims(test_X, 3), test_y, batch_size=32)

### Model Architectures

In [6]:
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as l

#### CNN Architecture 1 (Simple/Baseline)

In [22]:
model_1 = Sequential()

model_1.add(l.Flatten())
model_1.add(l.Dense(64, activation='tanh'))
#model_1.add(l.Dropout(0.3)) # 0.5
model_1.add(l.Dense(1, activation='sigmoid'))

#### CNN Architecture 2

In [21]:
model_2 = Sequential()

model_2.add(l.Conv2D(32, 3, 3, activation='tanh', input_shape=(128, 96, 1)))
model_2.add(l.Conv2D(64, 3, 3, activation='tanh'))
model_2.add(l.MaxPooling2D(pool_size=(2,2)))
#model_2.add(l.Dropout(0.15)) # 0.25
 
model_2.add(l.Flatten())
model_2.add(l.Dense(64, activation='tanh'))
#model_2.add(l.Dropout(0.3)) # 0.5
model_2.add(l.Dense(1, activation='sigmoid'))

#### CNN Architecture 3

In [ ]:
model_3 = Sequential()

model_3.add(l.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 128, 96, 1)))
model_3.add(l.Conv2D(64, (3, 3), activation='relu'))
model_3.add(l.MaxPooling2D(pool_size=(2, 2)))
model_3.add(l.Dropout(0.25))
model_3.add(l.Flatten())
model_3.add(l.Dense(128, activation='relu'))
model_3.add(l.Dropout(0.5))
model_3.add(l.Dense(11, activation='softmax'))

In [19]:
#model_1.summary()
model_2.summary()
#model_3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 42, 32, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2240)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                143424    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 162,305
Trainable params: 162,305
Non-trainable params: 0
________________________________________________

### Model Training

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='loss', mode='min', verbose=0, save_best_only=True)

train_X_r = np.reshape(train_X, (168, 128, 96, 1))
test_X_r = np.reshape(test_X, (50, 128, 96, 1))

#### Categorical CNN

In [ ]:
lr = 0.04
ba = 32
epoch = 30

model_2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_2.fit(train_X_r, train_y, 
          shuffle=True,
          epochs=epoch, 
          batch_size=ba, 
          validation_data=(test_X_r, test_y), 
          verbose=1,
          callbacks=[es, mc])

#### Regression CNN

In [30]:
lr = 10
ba = 32
epoch = 300

model_2.compile(loss='mean_absolute_error',
              optimizer='SGD',
              metrics=['mean_squared_error'])

model_2.fit(train_X_r, train_y, 
            shuffle=True,
            epochs=epoch, 
            batch_size=ba, 
            validation_data=(test_X_r, test_y), 
            verbose=1,
            callbacks=[es, mc])

Train on 168 samples, validate on 50 samples
Epoch 1/300
168/168 [==============================] - 1s 3ms/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 2/300
168/168 [==============================] - 0s 433us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 3/300
168/168 [==============================] - 0s 374us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 4/300
168/168 [==============================] - 0s 326us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 5/300
168/168 [==============================] - 0s 309us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 6/300
168/168 [==============================] - 0s 421us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_l

Epoch 98/300
168/168 [==============================] - 0s 279us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 99/300
168/168 [==============================] - 0s 285us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 100/300
168/168 [==============================] - 0s 297us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 101/300
168/168 [==============================] - 0s 332us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 102/300
168/168 [==============================] - 0s 338us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 103/300
168/168 [==============================] - 0s 297us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_er

168/168 [==============================] - 0s 279us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 195/300
168/168 [==============================] - 0s 332us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 196/300
168/168 [==============================] - 0s 427us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 197/300
168/168 [==============================] - 0s 267us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 198/300
168/168 [==============================] - 0s 261us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 199/300
168/168 [==============================] - 0s 303us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909


168/168 [==============================] - 0s 297us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 291/300
168/168 [==============================] - 0s 303us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 292/300
168/168 [==============================] - 0s 380us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 293/300
168/168 [==============================] - 0s 291us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 294/300
168/168 [==============================] - 0s 303us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 295/300
168/168 [==============================] - 0s 291us/sample - loss: 0.0909 - mean_squared_error: 0.0909 - val_loss: 0.0909 - val_mean_squared_error: 0.0909


#### Regression CNN with Data Augmentation

In [ ]:
epoch = 300

model.compile(loss='mean_absolute_error',
              optimizer=Adam(learning_rate=0.04),
              metrics=['mean_absolute_error', 'mean_squared_error'])

model.fit_generator(generator, 
                    shuffle=True,
                    epochs=epoch, 
                    verbose=1,
                    callbacks=[es, mc])

### Explore Results

In [ ]:
print(test_y[0])
plt.imshow(test_X[0], cmap='gray')

In [ ]:
test_X_r = np.expand_dims(test_X, 3).astype('float')

In [27]:
preds = model_2.predict(test_X_r)
preds